In [ ]:
#useful for debugging:
#from IPython.core.debugger import set_trace

import os
import zipfile
from tqdm.notebook import tqdm
import shutil
import time
from IPython.display import clear_output

class ProgressZipFile:
    """ProgressFile: simple class to work as a file-like object wrapper that can be used 
    to extract a ZipFile while at the same time updating a progress bar.
    """
   
    def __init__(self, zipfile, pbar_file, pbar_total):
        self.zipfile = zipfile
        self.file = None
        self.progress_file = 0
        self.progress_total = 0
        self.pbar_file = pbar_file
        self.pbar_total = pbar_total

    def close(self):
        #makes sure the bar is at exactly 100%
        self.file.seek(0, os.SEEK_END)
        self.pbar_file.n = self.file.tell()
        self.pbar_total.n = (self.pbar_total.n - self.progress_file) + self.file.tell()
        
        #notify tqdm that it should refresh (without forcing)
        self.pbar_file.update(0)
        self.pbar_total.update(0)
        
        self.file.close()
        
    def read(self, size):
        buf = self.file.read(size)
        self.progress_file += len(buf)
        self.progress_total += len(buf)
        self.pbar_file.update(len(buf))
        self.pbar_total.update(len(buf))

        return buf
        
    def readline(self, size):
        buf = self.file.readline(size)
        self.progress_file += len(buf)
        self.progress_total += len(buf)
        self.pbar_file.update(len(buf))
        self.pbar_total.update(len(buf))
        
        return buf   
        
    def set_active_zipinfo(self, zipinfo):
        if not self.file is None:
            self.close()
        self.zipinfo = zipinfo
        self.file = self.zipfile.open(zipinfo, mode='r')
        self.progress_file = self.file.tell()
        self.pbar_file.set_description('Current file: ' + zipinfo.filename)
        self.pbar_file.reset(total=zipinfo.file_size)
        self.pbar_file.refresh()
    
def getzipfoldersize(zipfile):
    '''
    Returns the total uncompressed size of all files in the zip file
    '''

    #get list of ZipInfo objects containing all entries in the zip file
    zipinfo = zipfile.infolist()

    total = 0
    #sum all file_size properties of all entries in zip file
    for file in zipinfo:
        total += file.file_size

    return total

def restore_timestamps(zfile, destination_path):
    infolist = zfile.infolist()
    for file in infolist:
            
        fullpath = os.path.join(destination_path, file.filename)
        
        #create dt object from stored date_time
        date_time = time.mktime(file.date_time + (0, 0, -1))
        
        # update date_time in file system
        os.utime(fullpath, (date_time, date_time))

#if use_zipname_as_subfolder is set to True, the files will be unzipped to a subfolder 
# using the (base)name of the zipfile, e.g. /path/zipfile/
use_zipname_as_subfolder = True #

source_path = '~/AmsterdamUMCdb-v1.0.2.zip' #path to zip file
destination_path = '~/'

#expands ~ for *nix environments
source_path = os.path.expanduser(source_path)
destination_path = os.path.expanduser(destination_path)

#extract file name of zip file
zfilename = os.path.basename(source_path)

#if True unzip the files to a subfolder using the (base)name of the zipfile, e.g. /path/zipfile/
if use_zipname_as_subfolder:
    
    basefoldername = os.path.splitext(zfilename)[0]
    destination_path = os.path.join(destination_path, basefoldername)

zfile = zipfile.ZipFile(source_path, 'r')
zinfo = zfile.infolist()

#get total uncompressed size of all files in the zip file
zfoldersize = getzipfoldersize(zfile)

#create two tqdm progress bars
pbar_total = tqdm(total=zfoldersize, initial=0, desc='Unzipping ' + zfilename, 
                      dynamic_ncols=True, unit_scale=1, unit='Bytes',
                      leave=True)

pbar_file = tqdm(total=0, desc='Current file:', dynamic_ncols=True, unit_scale=1, unit='Bytes',
                     leave=True)

pzfile = ProgressZipFile(zfile, pbar_file, pbar_total) #create a ProgressZipFile for showing progress

#extract all files from zipfile using ZipInfo
for fileinfo in zinfo:
    #information from zipfile
    filesize = fileinfo.file_size
    filename = fileinfo.filename
    
    #determine destination path
    filename_only = os.path.split(filename)[1]
    path_only = os.path.split(filename)[0]
    dest_filepath = os.path.join(destination_path, path_only)
    dest_filename = os.path.join(destination_path, filename)
    
    #create folders if they not already exist at destination
    if not os.path.exists(dest_filepath):
        os.makedirs(dest_filepath)

    #continue to the next entry if zipinfo object is a directory
    if os.path.isdir(dest_filename): 
        continue

    #changes the active zipinfo for the ProgressZipFile object to read entry from zipfile and update tqdm objects
    pzfile.set_active_zipinfo(fileinfo)
    
    #open/create a writable destination file (based on location if zipfile and zipinfo object)
    dest_file = open(dest_filename, 'wb')

    #copies the file-like objects ( ProgressZipFile -> File)
    shutil.copyfileobj(pzfile, dest_file)
    
    #clean up
    dest_file.close()
       
    #sets the progress status to finished when the last file has been processed 
    if fileinfo.filename == zinfo[-1].filename: 
        pbar_total.close()
        pbar_file.close()

#clean up
restore_timestamps(zfile, destination_path)
zfile.close()
print("Done.")